# Test running python in the background

Let's start by creating the resources we are going to need: a database and a container.

In [15]:
# jupyter nbconvert --to script InsertLoop.ipynb

import os
import time
import azure.cosmos
import azure.cosmos.cosmos_client as cosmos_client
from runpy import run_path

credentials = run_path("/home/cosmosuser/notebooks/credentials.py")

db_name = "mybgtest"
container_name = "myidcltest"

HOST = credentials['HOST']
MASTER_KEY = credentials['MASTER_KEY']
# with cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} ) as cosmos_client_id:
cosmos_client_id = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
# Create the database if it doesn't exist
try:
    database = cosmos_client_id.create_database_if_not_exists(db_name)
    print('Database "' + db_name + '" created')
    time.sleep(2)
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code == 409:
       print('Database already exists')
    else: 
        raise

# Reset the container
database_client = cosmos_client_id.get_database_client(db_name)
containers = database_client.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database_client.delete_container(container_name)
    print('Container "' + container_name + '" dropped')
database_client.create_container(container_name, azure.cosmos.PartitionKey(path="/id"))
print('Container "' + container_name + '" created')

Database "mybgtest" created
Container "myidcltest" created


### Importing documents

We will need some test data to work with, so we import 10,000 documents. Each document contains 2 fields: `field1` which has a random string value and `field2` which has a random integer value:

```
{
  "field1":"Garry75",
  "field2":405
}
```

This is going to take some time, so feel free to get a coffee ☕ in the meantime!

In [16]:
import urllib.request  as urllib2
import json
import uuid


container_client = database_client.get_container_client(container_name)

for i in range(5):
    req = urllib2.Request("http://api.open-notify.org/iss-now.json")
    response = urllib2.urlopen(req)

    obj = json.loads(response.read())
    # {"iss_position": {"longitude": "-151.4464", "latitude": "-51.0588"}, "timestamp": 1579293714, "message": "success"}
    print(obj)
 
    if obj['message'] == 'success':
        new_item = { 'id': str(uuid.uuid4()), 'timestamp': obj['timestamp'], 'latitude': obj['iss_position']['latitude'], 'longitude': obj['iss_position']['longitude'] }
        container_client.create_item(new_item)

        print (obj['timestamp'])
        print (obj['iss_position']['latitude'], obj['iss_position']['latitude'])
        
        time.sleep(5)
    else:
        print ('API failure: ', json.dumps(obj))



{'iss_position': {'latitude': '8.1635', 'longitude': '111.9993'}, 'message': 'success', 'timestamp': 1579381423}
1579381423
8.1635 8.1635
{'iss_position': {'latitude': '7.9109', 'longitude': '112.1834'}, 'message': 'success', 'timestamp': 1579381428}
1579381428
7.9109 7.9109
{'iss_position': {'latitude': '7.6330', 'longitude': '112.3856'}, 'message': 'success', 'timestamp': 1579381434}
1579381434
7.6330 7.6330
{'iss_position': {'latitude': '7.3802', 'longitude': '112.5692'}, 'message': 'success', 'timestamp': 1579381439}
1579381439
7.3802 7.3802
{'iss_position': {'latitude': '7.1273', 'longitude': '112.7525'}, 'message': 'success', 'timestamp': 1579381444}
1579381444
7.1273 7.1273


### Testing a first query

Now that we have our test data, time to issue some queries! We start by fetching all documents with a `field1` value of `Mario86`:

In [6]:
query = 'SELECT * FROM c'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 10 result(s)
Time elapsed: 71 ms
Request charge: 2.53 RUs


In [18]:
cosmos_client_id.delete_database(db_name)

CosmosResourceNotFoundError: Status code: 404
{"code":"NotFound","message":"Message: {\"Errors\":[\"Resource Not Found\"]}\r\nActivityId: 2bbe00fa-af2f-40d5-80b5-21027c0dd7e0, Request URI: /apps/fc748544-4998-4e48-8f0f-6b95ff54644f/services/a8da453c-08a3-4bbc-81cc-f67e796ac2b4/partitions/590f867e-9a99-418a-b1b8-1b9600969cc9/replicas/132236539267892150s, RequestStats: \r\nRequestStartTime: 2020-01-18T21:05:08.2897079Z, RequestEndTime: 2020-01-18T21:05:08.2897079Z,  Number of regions attempted:1\r\nResponseTime: 2020-01-18T21:05:08.2897079Z, StoreResult: StorePhysicalAddress: rntbd://10.0.0.20:11000/apps/fc748544-4998-4e48-8f0f-6b95ff54644f/services/a8da453c-08a3-4bbc-81cc-f67e796ac2b4/partitions/590f867e-9a99-418a-b1b8-1b9600969cc9/replicas/132236539267892150s, LSN: 32, GlobalCommittedLsn: 32, PartitionKeyRangeId: , IsValid: True, StatusCode: 404, SubStatusCode: 0, RequestCharge: 1, ItemLSN: -1, SessionToken: -1#32, UsingLocalLSN: False, TransportException: null, ResourceType: Database,